# 十一、基于陷门置换的公钥加密系统
## 公钥加密系统：定义和安全性
定义部分可以参考第九章最后一节的内容，没有任何变化。

选择明文攻击的安全性：系统将公钥发送给攻击者，攻击者发送两条明文给系统，系统返回密文，攻击者无法判断密文所对应的明文。

对于对称加密系统，第一周就已经证明了，一次性密钥安全不等于选择明文攻击安全（对应多次密钥）。但对于公钥加密系统而言，由于攻击者具有公钥可以自行加密，>实质上具备一次性密钥安全的系统就已经具备了选择明文攻击。

选择密文攻击的安全性：攻击者使用公钥进行加密的密文发送个系统，系统会返回对应的明文。之后攻击者再发送两条明文，系统返回其中一条对应的密文。之后攻击者>可以继续发送不同于返回密文的其它任意密文，最终目标是攻击者依旧无法判断密文对应的明文。

对称加密系统解决CCA攻击的方式是认证加密，即攻击者无法创造新的密文，从而隐含也就避免了CCA攻击。而在公钥加密系统的情况下，攻击者可以用公钥创造新的密文>，系统需要直接解决CCA攻击的问题。

## 构造公钥加密系统
陷门函数（Trapdoor function，TDF）包括三个部分：1）生成器G()，生成一对公钥（pk）和私钥（sk）。2）函数F(pk, )，使用公钥将X投射到Y。3）函数F<sup>-1</sup>(sk, c)，使用私钥将Y投射回X。同样的x，在F和F<sup>-1</sup>计算的过程中能够还原。

一个安全的陷门函数，给定pk和y，无法找到有效的算法求x。即给定y，务必需要私钥sk才能还原x。换言之，F是一个单向函数。

安全的公钥加密系统可以由：1）安全的陷门函数（G(), F, F<sup>-1</sup>）。2）具备认证加密的对称加密系统（E<sub>s</sub>, D<sub>s</sub>）。3）随机预言的哈
希函数H。三个部分进行构造。构造过程如下：

加密部分E(pk, m)：y = F(pk, x), k = H(x), c = E<sub>s</sub>(k, m), output = (y, c)。

解密部分D(sk, (y, c))：x = F-1(sk, y), k = H(x), m = D<sub>s</sub>(k, c), output(m)。

事实上，上面说的这个是ISO标准。需要注意的是，不要直接使用陷门函数进行加密。首先，这是一个确定性的加密系统，显然不可能具备语义安全。同时，这个系统也会
面临许多其它的攻击。

## RSA陷门置换
公钥加密系统需要安全的陷门函数，RSA陷门置换是最为广泛应用的TDF，具体构造如下：

G()：随机生成两个长度大约为1024位的素数p、q，令N = p * q。在集合N上，抽取整数e和d，使得e * d = 1 (mod φ(N))。公钥pk = e (mod N)，私钥sk = d (mod N)。

F(pk, m)：在星号集合N上随机抽取x，计算y = RSA(x) = x<sup>e</sup> (mod N)。

F<sup>-1</sup>(sk, y)：= y<sup>d</sup> = RSA(x)<sup>d</sup> = x<sup>ed</sup> = x<sup>(kφ(N)+1)</sup>=x<sup>kφ(N)</sup> * x = x。

注意p和q的大小约等于N<sup>1/2</sup>，因此在N非常大时（p,q是1024位，则N是2048位，大约是十进制下的600数位），φ(N)=N-p-q+1，约等于N。也即是说，星号集合>的元素数量，基本接近于N的集合。在集合N上抽取的整数，很大可能也落在星号集合内，无需求解星号集合，整数来自星号集合意味着在集合N内具有模逆元。故给定e，>很容易利用扩展欧几里得算法，得到d。

RSA仅仅是一个陷门函数，不是加密函数，需要集成到上节所述的ISO标准中去使用。如果直接使用RSA陷门函数进行加密，十分不安全。

RSA通常用来在建立会话的过程中交换通信密钥。考虑一个64位的密钥k，如果直接使用c = k<sup>e</sup>，则将k分解的话，令k = k<sub>1</sub> * k<sub>2</sub>，k<sub>1</sub>和k<sub>2</sub>有20%的可能小于2<sup>32</sup>。将每一种k<sub>1</sub>及其计算出的c/k<sub>1</sub><sup>e</sup>值写进一个哈希表，然后查看每一种
可能的k<sub>2</sub><sup>e</sup>是否在表内。这样所需要花费的时间大约是2<sup>40</sup>，远小于2<sup>64</sup>。

## PKCS1
在实际中常用的也不是ISO标准，而是PKCS1（Public Key Cryptography Standards）。RSA用于会话密钥交换，常见于建立会话的时候。

## RSA是否确实为单向置换函数
对于RSA函数，在只知道e和y的情况下，要计算x，就是要计算x的模e次方根，如前所述，这里需要首先对N做因子分解为素数p和q，然后再分别计算x在素数p和q上的模e次
方根。后者简单，而前者十分困难。

某些加速RSA的办法会导致不安全。如将私钥d设小。已经证明的情况是，如果d小于N<sup>0.292</sup>时，该私钥可以被破解。

## 实践中的RSA
加速RSA的办法，可以将公钥e设小，推荐的一个值是e = 65537 = 2<sup>16</sup> + 1。这样加密过程就变快了，而解密过程变慢。

实践中，RSA依然会受到旁路攻击。另外OpenSSL之前曝出的一个漏洞，是机器在启动时，熵值过低，导致p和q的选取不够随机，十分容易被破解。


# 十二、基于迪菲-赫尔曼协议的公钥加密系统
## ElGamal公钥加密系统
公钥加密主要用于如HTTP协议中的通信密钥交换过程，交换了通信密钥之后，双方可以用对称加密的形式进行通信。

但有时，对于某些非交互场景:1）如Email发送，公钥加密也用来直接传输信息。2）加密文件系统的共享，只需要将加密的密钥用共享者的公钥进行一道加密，封装到头>文件。3）密钥托管，企业中每个人的加密文件，需要将加密密钥用托管方的公钥进行加密，而具备高权限的托管方在需要时，可以用私钥解密出加密密钥，从而查看特定
对象的加密文件。

回顾一下迪菲-赫尔曼协议，结合数论的知识，g是关于p的星号集合上的一个迭代器，因此g<sup>a</sup>，g<sup>b</sup>，g<sup>ab</sup>均为星号集合中的元素。用于
公钥加密系统时，在交换信息时，B直接将信息B和用A<sup>b</sup> = g<sup>ab</sup>加密的通信密钥发送给A，之后A通过计算B<sup>a</sup>得到g<sup>ab</sup>，从而
解密出通信密钥。

ElGamal公钥加密系统由三个部分组成：1）秩为n的循环组。2）具备认证加密的对称加密系统（E<sub>s</sub>, D<sub>s</sub>）。3）函数H：G<sup>2</sup> -> K。构>造过程如下：

在G上找到一个迭代器g以及集合N上的一个随机数a。sk = a，pk = (g, h=g<sup>a</sup>)。

加密部分E(pk = (g, h), m)：集合N上获得随机数b，u = g<sup>b</sup>，v = h<sup>b</sup>，k = H(u, v), c = E<sub>s</sub>(k, m), output = (u, c)。

解密部分D(sk = a, (u, c))：v = u<sup>a</sup>, k = H(u, v), m = D<sub>s</sub>(k, m), output = m。

## ElGamal的安全性
给定g，g<sup>a</sup>，g<sup>b</sup>，没有有效的算法可以计算g<sup>ab</sup>，这是迪菲-赫尔曼协议安全性的基础。对于ElGamal而言，这一假设则变为，给定g，g<sup>a</sup>，g<sup>b</sup>，无法分辨H(g<sup>b</sup>, g<sup>ab</sup>)与随机数R，简写为HDH。

只要对称加密系统具备认证加密，哈希函数是随机预言，同时HDH假设成立，ElGamal公钥加密就具备语义安全。但需要ElGamal具备选择密文攻击安全，则要求更严格的假
设，成为IDH（Interactive Diffie-Hellman）。

## 更安全的ElGamal变形
一般的ElGamal需要IDH才能保证选择密文攻击安全，而twin ElGamal构造可以解决这个问题，只需要HDH，就可以保证选择密文攻击安全。其构造过程如下：

在G上找到一个迭代器g以及集合N上的两个随机数a<sub>1</sub>，a<sub>2</sub>。sk = (a<sub>1</sub>, a<sub>2</sub>)，pk = (g, h<sub>1</sub>=g<sup>a1</sup?, h<sub>2</sub>=g<sup>a2</sup>)。

加密部分E(pk = (g, h<sub>1</sub>, h<sub>2</sub>), m)：集合N上获得随机数b，u = g<sup>b</sup>，v<sub>1</sub> = h<sub>1</sub><sup>b</sup>，v<sub>2</sub> = h<sub>2</sub><sup>b</sup>，k = H(u, v<sub>1</sub>, v<sub>2</sub>), c = E<sub>s</sub>(k, m), output = (u, c)。

解密部分D(sk = (a<sub>1</sub>，a<sub>2</sub>), (u, c))：v<sub>1</sub> = u<sup>a1</sup>, v<sub>2</sub> = u<sup>a2</sup> k = H(u, v<sub>1</sub>, v<sub>2</sub>), m = D<sub>s</sub>(k, m), output = m。

## 更加泛化的主题
无论陷门函数，还是迪菲-赫尔曼协议，共同点在单向函数，即不容易求逆的函数。密钥交换，正是因为单向函数才成为可能。

一个安全的伪随机生成器，本身是一个单向函数，但无法用于密钥交换。

离散对数（DH协议）也是伪随机生成器，并可以用于密钥交换。

RSA函数，陷门。

## 告别
回顾和展望，略。